<a href="https://colab.research.google.com/github/Brennan-Richards/Research-Recommendation/blob/main/Academic_Research_Recommendation_Fine_Tuning_GPT_2_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 32.6 MB/s 
     |████████████████████████████████| 880 kB 54.1 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 61.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=160f828f15d46c6a88b53c8ff303c3e50e262b9482086a8538d09f4c31e6e8a8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 29.4 MB/s 
     |████████████████████████████████| 136 kB 52.8 MB/s 
     |████████████████████████████████| 1.1 MB 64.7 MB/s 
     |████████████████████████████████| 212 kB 70.2 MB/s 
     |████████████████████████████

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
!pip install huggingface_hub

In [ ]:
import tensorflow as tf
import os

In [ ]:
from transformers import DataCollator, DefaultDataCollator, DataCollatorWithPadding
from transformers import AutoTokenizer, GPT2Tokenizer, TFGPT2LMHeadModel, GPT2LMHeadModel
from transformers import AdamW, AdamWeightDecay
from datasets import load_dataset

from transformers import TFAutoModelForSequenceClassification, TFAutoModelForCausalLM, TFAutoModel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/')
# os.chdir('content/drive/MyDrive/CSC561')
os.chdir('content/drive/Shareddrives/CSC561 Finals')

In [ ]:
!pwd

/content/drive/Shareddrives/CSC561 Finals


### GPT-2 Fine-tune for Topics

In [ ]:
import pandas as pd

In [ ]:
# df1 = pd.read_csv('tags_by_user.csv').dropna().drop_duplicates()
# df1.describe()

In [ ]:
# df1.to_csv('tags_by_user.csv', index=False)


In [ ]:
model_name = 'gpt2'
data_file = "permuted_tags_by_user.csv"

In [ ]:
n_rows = len(pd.read_csv(data_file))

print(f"Dataset has {n_rows}")

topic_seqs = load_dataset("csv", data_files=data_file, split=f"train[:{ str(n_rows) }]")
topic_seqs = topic_seqs.train_test_split(test_size=0.2)

Dataset has 17318


Using custom data configuration default-0421c87e9139fabe


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0421c87e9139fabe/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


In [ ]:
topic_seqs

DatasetDict({
    train: Dataset({
        features: ['tag_list'],
        num_rows: 13854
    })
    test: Dataset({
        features: ['tag_list'],
        num_rows: 3464
    })
})

In [ ]:
topic_seqs["train"][290]

{'tag_list': 'bianconi barabasi model'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
topic_seqs = topic_seqs.flatten()

In [ ]:
def preprocess_function(examples):
    # print(examples)
    return tokenizer(examples["tag_list"], truncation=True)

In [ ]:
tokenized_topic_seqs = topic_seqs.map(
    preprocess_function,
    remove_columns=topic_seqs["train"].column_names,
)

  0%|          | 0/13854 [00:00<?, ?ex/s]

  0%|          | 0/3464 [00:00<?, ?ex/s]

In [ ]:
# tokenized_topic_seqs["train"][0]

In [ ]:
block_size=128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, though you could add padding instead if the model supports it
    # In this, as in all things, we advise you to follow your heart
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_topic_seqs.map(group_texts, batch_size=1000, batched=True, num_proc=4) #, batched=True, num_proc=4)

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#3:   0%|          | 0/4 [00:00<?, ?ba/s]

#2:   0%|          | 0/4 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")

In [ ]:
tf_train_set = lm_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    dummy_labels=True,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = lm_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    dummy_labels=True,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
!git config --global credential.helper store

In [ ]:
# from transformers.keras_callbacks import PushToHubCallback
# from tensorflow.keras.callbacks import TensorBoard

# # model_name = model_checkpoint.split("/")[-1]
# push_to_hub_model_id = f"brennan-richards/{model_name}-finetuned-academic-topics"

# tensorboard_callback = TensorBoard(log_dir="./clm_model_save/logs")

# push_to_hub_callback = PushToHubCallback(
#     output_dir="./clm_model_save",
#     tokenizer=tokenizer,
#     hub_model_id=push_to_hub_model_id,
# )


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=5)

Epoch 1/5
187/187 [==============================] - 143s 669ms/step - loss: 4.2121 - val_loss: 3.4634
Epoch 2/5
187/187 [==============================] - 128s 683ms/step - loss: 3.3499 - val_loss: 2.9760
Epoch 3/5
187/187 [==============================] - 128s 687ms/step - loss: 3.0020 - val_loss: 2.7735
Epoch 4/5
187/187 [==============================] - 128s 686ms/step - loss: 2.8367 - val_loss: 2.6828
Epoch 5/5
187/187 [==============================] - 128s 686ms/step - loss: 2.7480 - val_loss: 2.6372


In [ ]:
# Save model temporarily as a TF model
model.save_pretrained(f'{model_name}-finetuned-academic-topics')

In [ ]:
# Convert TF model to Pytorch

pt_model = GPT2LMHeadModel.from_pretrained(f'{model_name}-finetuned-academic-topics', from_tf=True)
# pt_model.save_pretrained(f"{model_path}_pt")

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


In [ ]:
# Push Pytorch model to the hub

pt_model.push_to_hub(f"brennan-richards/{model_name}-finetuned-academic-topics")

In [ ]:
from transformers import pipeline

generator = pipeline(task="text-generation", model=f"{model_name}-finetuned-academic-topics", tokenizer=tokenizer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-finetuned-academic-topics.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
input = "deep learning, transformers, computer vision"
max_len = 20
print(max_len)
output_list = generator(input, num_return_sequences=1, max_length=max_len ) # Set max length as the len of the input + (len of largest token in dataset)
print("OUTPUT: ", output_list)
recommendation = output_list[0]['generated_text'].split(', ')[len(input.split(", "))]
print("RECOMMENDATION: ", recommendation)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


20
OUTPUT:  [{'generated_text': 'deep learning, transformers, computer vision, scientific knowledge management, parietal cortex, math, info'}]
RECOMMENDATION:  scientific knowledge management


In [ ]:
# Parse the suggestion out of the generator pipeline
recommendation

'science studies'

In [ ]:
# model.save_pretrained('distilgpt2-ft-on-topics-5-1')

### OpenAlex

In [ ]:
import requests

In [ ]:
# Return a list of concepts related to the search term
url = f"https://api.openalex.org/concepts?search={ recommendation }"
print(url)
r_json = requests.get(url).json()

Interesting attributes:
- Related concepts for each concept returned

In [ ]:
r_json['results'][0].keys()

In [ ]:
top_5 = r_json["results"][0:5]
filtered = [(c['display_name'], c['works_api_url']) for c in top_5]
filtered

In [ ]:
# Example call to Works API URL
# NOTE: Sort by relevance
wr_json = requests.get(filtered[0][1]).json()

In [ ]:
wr_json['results'][0].keys()

In [ ]:
# NOTE: Will need URL
top_5_works = wr_json["results"][0:5]
filtered_w = [(w['title'], w['host_venue']['url']) for w in top_5_works]
filtered_w